In [7]:
import pandas as pd
dataset=pd.read_csv("./Project2Data.txt",delimiter=",",header=None,)
hidden_states = ['sunny','rainy','foggy']

In [8]:
def calculate_transmission_probabilities():
    previous_row = []
    sunny_sunny , sunny_rainy , sunny_foggy = 0,0,0
    rainy_rainy , rainy_sunny , rainy_foggy = 0,0,0
    foggy_foggy , foggy_sunny , foggy_rainy = 0,0,0
    for index, row in dataset.iterrows():
        if(len(previous_row) != 0):
            if(previous_row[0] == "sunny" and row[0] == "sunny"):
                sunny_sunny = sunny_sunny + 1
            elif(previous_row[0] == "sunny" and row[0] == "foggy"):
                sunny_foggy = sunny_foggy + 1
            elif(previous_row[0] == "sunny" and row[0] == "rainy"):
                sunny_rainy = sunny_rainy + 1
            elif(previous_row[0] == "rainy" and row[0] == "rainy"):
                rainy_rainy = rainy_rainy + 1
            elif(previous_row[0] == "rainy" and row[0] == "foggy"):
                rainy_foggy = rainy_foggy + 1
            elif(previous_row[0] == "rainy" and row[0] == "sunny"):
                rainy_sunny = rainy_sunny + 1
            elif(previous_row[0] == "foggy" and row[0] == "foggy"):
                foggy_foggy = foggy_foggy + 1
            elif(previous_row[0] == "foggy" and row[0] == "rainy"):
                foggy_rainy = foggy_rainy + 1
            elif(previous_row[0] == "foggy" and row[0] == "sunny"):
                foggy_sunny = foggy_sunny + 1
            previous_row = row
        else:
            previous_row = row

    prob_sunny_sunny = sunny_sunny / (sunny_sunny + sunny_rainy + sunny_foggy)
    prob_sunny_rainy = sunny_rainy / (sunny_sunny + sunny_rainy + sunny_foggy)
    prob_sunny_foggy = sunny_foggy / (sunny_sunny + sunny_rainy + sunny_foggy)

    prob_rainy_rainy = rainy_rainy / (rainy_rainy + rainy_sunny + rainy_foggy)
    prob_rainy_sunny = rainy_sunny / (rainy_rainy + rainy_sunny + rainy_foggy)
    prob_rainy_foggy = rainy_foggy / (rainy_rainy + rainy_sunny + rainy_foggy)

    prob_foggy_foggy = foggy_foggy / (foggy_foggy + foggy_sunny + foggy_rainy)
    prob_foggy_sunny = foggy_sunny / (foggy_foggy + foggy_sunny + foggy_rainy)
    prob_foggy_rainy = foggy_rainy / (foggy_foggy + foggy_sunny + foggy_rainy)
    
    hidden_states = ['sunny','rainy','foggy']
    aij_df=pd.DataFrame(columns = hidden_states, index = hidden_states)
    aij_df.loc[hidden_states[0]] = [prob_sunny_sunny,prob_sunny_rainy,prob_sunny_foggy]
    aij_df.loc[hidden_states[1]] = [prob_rainy_sunny,prob_rainy_rainy,prob_rainy_foggy]
    aij_df.loc[hidden_states[2]] = [prob_foggy_sunny,prob_foggy_rainy,prob_foggy_foggy]
    return aij_df

aij_df = calculate_transmission_probabilities()

print("aij Matrix:\n", aij_df)

aij Matrix:
           sunny      rainy     foggy
sunny  0.795132  0.0547667  0.150101
rainy  0.195918   0.579592   0.22449
foggy  0.206897   0.291188  0.501916


In [9]:
def calculate_emisiion_probability():
    sunny_yes, sunny_no = 0 ,0
    rainy_yes, rainy_no = 0, 0
    foggy_yes, foggy_no = 0, 0

    for index, row in dataset.iterrows():
        if(row[0] == "sunny" and row[1] == "yes"):
            sunny_yes = sunny_yes + 1
        elif(row[0] == "sunny" and row[1] == "no"):
            sunny_no = sunny_no + 1
        elif(row[0] == "rainy" and row[1] == "yes"):
            rainy_yes = rainy_yes + 1
        elif(row[0] == "rainy" and row[1] == "no"):
            rainy_no = rainy_no + 1
        elif(row[0] == "foggy" and row[1] == "yes"):
            foggy_yes = foggy_yes + 1
        elif(row[0] == "foggy" and row[1] == "no"):
            foggy_no = foggy_no + 1

    prob_sunny_yes = sunny_yes / (sunny_yes + sunny_no)
    prob_sunny_no = sunny_no / (sunny_yes + sunny_no)

    prob_rainy_yes = rainy_yes / (rainy_yes + rainy_no)
    prob_rainy_no = rainy_no / (rainy_yes + rainy_no)

    prob_foggy_yes = foggy_yes / (foggy_yes + foggy_no)
    prob_foggy_no = foggy_no / (foggy_yes + foggy_no)

    observable_states = ['yes','no']
    hidden_states = ['sunny','rainy','foggy']
    #bjk_df = pd.DataFrame(columns= hidden_states, index= observable_states)
    #bjk_df.loc[observable_states[0]] = [prob_sunny_yes , prob_rainy_yes , prob_foggy_yes]
    #bjk_df.loc[observable_states[1]] = [prob_sunny_no , prob_rainy_no , prob_foggy_no]
    
    bjk_df = pd.DataFrame(columns=observable_states , index=hidden_states )
    bjk_df.loc[hidden_states[0]] = [prob_sunny_yes , prob_sunny_no]
    bjk_df.loc[hidden_states[1]] = [prob_rainy_yes , prob_rainy_no]
    bjk_df.loc[hidden_states[2]] = [prob_foggy_yes , prob_foggy_no]
    return bjk_df

bjk_df = calculate_emisiion_probability()
print("bjk Matrix:\n",bjk_df)

bjk Matrix:
              yes        no
sunny  0.0850202   0.91498
rainy   0.812245  0.187755
foggy   0.302682  0.697318


In [10]:
import numpy as np

def forward_algorithum(visibleStateList):
    a = np.array(aij_df.values)
    b = np.array(bjk_df.values)
    initial_distribution=np.array((1,0,0))
    v = visibleStateList
    alpha = np.zeros((len(v), a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, v[0]]
    for t in range(1, len(v)):
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, v[t]]
    return alpha

def convert_to_stateindex(visibleStateList):
    resultVisibleStateList =[]
    for x in visibleStateList:
        if x == "yes":
            resultVisibleStateList.append(0)
        else:
            resultVisibleStateList.append(1)
    return resultVisibleStateList


visibleStateList = ["yes","yes","no","no"]
resultVisibleStateList = convert_to_stateindex(visibleStateList)
alpha = forward_algorithum(resultVisibleStateList)
print(alpha)
print("vt is [0,0,1,1], p(vt/theta): ",alpha[3][2])

[[0.08502024 0.         0.        ]
 [0.00574756 0.00378204 0.00386272]
 [0.00559073 0.00068185 0.00254557]
 [0.00467154 0.00027086 0.00158285]]
vt is [0,0,1,1], p(vt/theta):  0.0015828455867612503


In [11]:
def viterbi_algorithm(visibleStateList): 
    visbleStateListIndex = convert_to_stateindex(visibleStateList)
    forward_output = forward_algorithum(visbleStateListIndex)
    print(forward_output)
    hidden_state_output = []
    for i in range(len(forward_output)): 
        max_index = np.argmax(forward_output[i])
        hidden_state_output.append(hidden_states[max_index])
    return hidden_state_output

In [12]:
viterbi_algorithm(visibleStateList)

[[0.08502024 0.         0.        ]
 [0.00574756 0.00378204 0.00386272]
 [0.00559073 0.00068185 0.00254557]
 [0.00467154 0.00027086 0.00158285]]


['sunny', 'sunny', 'sunny', 'sunny']

In [14]:
visibleStateList2 = ["yes","yes","yes","yes"]
viterbi_algorithm(visibleStateList2)

[[0.08502024 0.         0.        ]
 [0.00574756 0.00378204 0.00386272]
 [0.00051949 0.00294974 0.00110494]
 [0.00010369 0.0016731  0.0003919 ]]


['sunny', 'sunny', 'rainy', 'rainy']